In [2]:
import numpy as np
from catboost import Pool, CatBoostRegressor

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_train = pd.read_csv('train.csv')

In [21]:
df_train.drop('Id', axis=1,inplace=True) 

In [13]:
categorical_features = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
        'Heating', 'HeatingQC', 'Electrical', 'KitchenQual',
        'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
        'GarageCond', 'PoolQC', 'Fence', 'MiscFeature',
        'SaleType', 'SaleCondition','Utilities','PavedDrive']

In [16]:
numerical_columns = [ 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
        'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold','LotFrontage', 'MasVnrArea',
                     'GarageYrBlt']

In [18]:
all_features = numerical_columns + categorical_features

In [22]:
df_train = df_train[all_features]

In [14]:
# initialize Pool
train_pool = Pool(df_train, 
                  df_train['SalePrice'], 
                  cat_features=df_train[])

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=6]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [10]:
train_pool

In [ ]:

test_pool = Pool(test_data, 
                 cat_features=[0,2,5]) 

# specify the training parameters 
model = CatBoostRegressor(iterations=2, 
                          depth=2, 
                          learning_rate=1, 
                          loss_function='RMSE')
#train the model
model.fit(train_pool)
# make the prediction using the resulting model
preds = model.predict(test_pool)
print(preds)